In [8]:
import networkx as nx
import pandas as pd

#Network to evaluate links
GMissingEdges = nx.read_gml("GraphMissingEdges.gml")
#Links to be evaluated
dfEdgesToEvaluate = pd.read_csv('edgesToEvaluate.csv')
lista = []
for idx, row in dfEdgesToEvaluate.iterrows( ):
    venue1 = row['venue1']
    venue2 = row['venue2']
    lista.append((venue1, venue2))

pred_adamic = list(nx.adamic_adar_index(GMissingEdges, lista))


criação de lista q armazenara os resultados do adamic, para extrair estatisticas

In [9]:
lista_lontitude = []
lista_latitude = []
lista_valor = []
lista_p = []

In [10]:
for u, v, p in pred_adamic:
    cate1 = GMissingEdges.nodes[u]['categories']
    lat1 = float(GMissingEdges.nodes[u]['latitude'])
    lon1 = float(GMissingEdges.nodes[u]['longitude'])
    
    
    cate2 = GMissingEdges.nodes[v]['categories']
    lat2 = float(GMissingEdges.nodes[v]['latitude'])
    lon2 = float(GMissingEdges.nodes[v]['longitude'])

    
    setCate1 = set(cate1.split(','))
    setCate2 = set(cate2.split(','))

    
    lista_valor.append(len(setCate1.intersection(setCate2)))
    lista_latitude.append(abs(lat1 - lat2))
    lista_lontitude.append(abs(lon1 - lon2))
    lista_p.append(p)


medindos os quartis pra usar de limiar pra escolher os link

In [11]:
import statistics
latitude_limite = statistics.quantiles(lista_latitude)
longitude_limite = statistics.quantiles(lista_lontitude)
intersec_limite = statistics.quantiles(lista_valor)
intersec_p = statistics.quantiles(lista_p)
print(latitude_limite)
print(longitude_limite)
print(intersec_limite)
print(intersec_p)

[0.007309974999998303, 0.02137869999999964, 0.05593520529999729]
[0.015445425499994059, 0.042002350000004185, 0.08602537652500075]
[0.0, 0.0, 1.0]
[0.0, 0.0, 0.2559830957079857]


In [12]:
listInference = []
for quantidade_grupo,lat,lon,p in zip(lista_valor,lista_latitude,lista_lontitude,lista_p):
    # usando quartil superior para os q estão relativamente pertos e mesmo grupo então deve haver um link
    if quantidade_grupo >= intersec_limite[2] and lat < latitude_limite[2] and lon < longitude_limite[2]:
        listInference.append(1)# n pode abaixar
    else:
        # usando quartil inferior para os q estão muitos perto então deve haver um link
        if  lat < latitude_limite[0] and  lon < longitude_limite[0]: 
            listInference.append(1) 
        else:
            if p > intersec_p[2]:
                listInference.append(1)
            else:
                listInference.append(0)

dfEdgesToEvaluate['link'] = listInference
dfEdgesToEvaluate.to_csv("teste_arestas.csv", columns=['linkID','link'],index=False)